In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Loading Data and Lexicon

In [ ]:
!pip install transformers
import pandas as pd

from datetime import datetime
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModel, BertModel
from sklearn.metrics.pairwise import cosine_similarity


import re  
from gensim.models import Word2Vec    #Import gensim Word2Fec
from sklearn.decomposition import PCA #Grab PCA functions
import numpy as np

#Plot helpers
import matplotlib.pyplot as plt

#Enable matplotlib to be interactive (zoom etc)
import ast

#new ones_collected by passing through the emobert
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity.csv")

# df['embedding'] = df['embedding'].tolist()
df['embedding'] = [ast.literal_eval(i) for i in df['embedding'].values.tolist()]
print(df.head())
print(df.columns)
df = df.dropna()

        word  ...                                          embedding
0   outraged  ...  [1.124068260192871, 0.7084425091743469, -0.467...
1  brutality  ...  [1.3678280115127563, 0.2582870125770569, -0.73...
2     hatred  ...  [1.1745389699935913, 0.8239633440971375, -0.41...
3    hateful  ...  [0.8323023915290833, 0.9238643646240234, -0.55...
4  terrorize  ...  [1.417137861251831, -0.09984711557626724, -1.0...

[5 rows x 4 columns]
Index(['word', 'emotion', 'emotion-intensity-score', 'embedding'], dtype='object')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9921 entries, 0 to 9920
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     9921 non-null   object 
 1   emotion                  9921 non-null   object 
 2   emotion-intensity-score  9921 non-null   float64
 3   embedding                9921 non-null   object 
dtypes: float64(1), object(3)
memory usage: 387.5+ KB


In [ ]:
Counter(df['word'].tolist())

Counter({'outraged': 1,
         'brutality': 2,
         'hatred': 4,
         'hateful': 4,
         'terrorize': 3,
         'infuriated': 2,
         'violently': 4,
         'furious': 2,
         'enraged': 1,
         'furiously': 1,
         'screwyou': 1,
         'murderer': 4,
         'fury': 3,
         'execution': 4,
         'angered': 2,
         'savagery': 2,
         'slaughtering': 5,
         'veryangry': 1,
         'assassinate': 2,
         'fuckoff': 1,
         'annihilation': 3,
         'rage': 1,
         'loathe': 2,
         'damnation': 3,
         'roadrage': 1,
         'fucktard': 1,
         'homicidal': 2,
         'furor': 3,
         'hostile': 4,
         'annihilate': 2,
         'murder': 5,
         'raging': 3,
         'sopissed': 1,
         'pissed': 4,
         'rape': 4,
         'explosive': 4,
         'obliterated': 2,
         'vengeful': 2,
         'ferocious': 3,
         'infuriates': 1,
         'killing': 7,
         'combativ

In [ ]:
# from sklearn.cluster import KMeans
# X = df['embedding']
# no_of_cluster= 8
# k_means_model=KMeans(n_clusters=no_of_cluster).fit(X)

In [ ]:
! pip install annoy -q

In [ ]:
from annoy import AnnoyIndex
import random

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

# super fast, will just mmap the file
t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_incremental.ann') 

True

In [ ]:
t.get_n_items()

18044

#Emotion Extraction Pipeline

In [ ]:
negations = ['never','not', 'no', 'didn', "didn't", 'didnt', 'didn t','doesn t', 'doesnt', "doesn't", "won't", 'won t', 'wont', "isn't","isnt","isn t", "aren't", 'aren t', 'arent', 'don t', "don't", 'dont',"haven't","haven t","havent",'weren t', "weren't", 'werent', "wasn't", 'wasn t', 'wasnt', 'wouldn t', "wouldn't", 'wouldnt', "can't", 'can t', 'couldn t',"couldn't", 'cant', 'cannot', 'couldnt', 'shouldnt', "shouldn't", 'shouldn t', 'neither', 'impossible', 'didn', 'wasn', 'weren', 'aren', 'don', 'doesn', 'couldn', 'shouldn', 'wouldn', 'won','nothing']

intensity_modifiers=      ['absolutely:B_INCR:0.9', 'almost:B_DECR:0.2', 'always:B_INCR:0.9', 'amazingly:B_INCR:0.8', 'awfully:B_INCR:0.8', 
      'completely:B_INCR:0.9', 'considerable:B_INCR:0.6', 'considerably:B_INCR:0.6', 'decidedly:B_INCR:0.6', 'deeply:B_INCR:0.9', 
      'effing:B_INCR:0.7', 'enormous:B_INCR:0.8', 'enormously:B_INCR:0.8', 'entirely:B_INCR:0.9', 'especially:B_INCR:0.8', 'exceptional:B_INCR:0.8', 
      'exceptionally:B_INCR:0.8', 'extreme:B_INCR:0.9', 'extremely:B_INCR:0.9', 'fabulously:B_INCR:0.8', 'flippin:B_INCR:0.6', 'flipping:B_INCR:0.6', 
      'frackin:B_INCR:0.6', 'fracking:B_INCR:0.6', 'frickin:B_INCR:0.7', 'fricking:B_INCR:0.7', 'friggin:B_INCR:0.7', 'frigging:B_INCR:0.7', 'fully:B_INCR:0.9', 
      'greatly:B_INCR:0.8', 'hella:B_INCR:0.8','bloody:B_INCR:0.8', 'highly:B_INCR:0.9', 'hugely:B_INCR:0.9', 'incredible:B_INCR:0.9', 'incredibly:B_INCR:0.9', 'intensely:B_INCR:0.9',
      'just enough:B_DECR:0.4', 'kind of:B_DECR:0.3', 'kinda:B_DECR:0.3', 'kindof:B_DECR:0.3', 'kind-of:B_DECR:0.3', 'less:B_DECR:0.3', 'little:B_DECR:0.3', 'major:B_INCR:0.6',
      'majorly:B_INCR:0.6', 'marginal:B_DECR:0.3', 'marginally:B_DECR:0.3', 'more:B_INCR:0.6', 'most:B_INCR:0.8', 'not much:B_DECR:0.2', 'occasional:B_DECR:0.6', 
      'occasionally:B_DECR:0.6', 'particularly:B_INCR:0.7', 'partly:B_DECR:0.3','partially:B_DECR:0.5', 'purely:B_INCR:0.9', 'quite:B_INCR:0.6', 'really:B_INCR:0.8', 'remarkably:B_INCR:0.9', 
      'slight:B_DECR:0.3', 'slightly:B_DECR:0.6','barely:B_DECR:0.7', 'so:B_INCR:0.8', 'somewhat:B_DECR:0.4', 'soo:B_INCR:0.9', 'sort of:B_DECR:0.4', 'sorta:B_DECR:0.4', 'sortof:B_DECR:0.4', 
      'sort-of:B_DECR:0.4', 'substantially:B_INCR:0.7', 'super:B_INCR:0.8', 'thoroughly:B_INCR:0.8', 'total:B_INCR:0.8', 'totally:B_INCR:0.9', 'tremendous:B_INCR:0.8', 
      'tremendously:B_INCR:0.8', 'truly:B_INCR:0.9', 'unbelievably:B_INCR:0.9', 'unusually:B_INCR:0.7', 'utter:B_INCR:0.8', 'utterly:B_INCR:0.8', 'very:B_INCR:0.8', 
      'not very:B_DECR:0.7']





In [ ]:
!pip install colour
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.core.display import display, HTML
from colour import Color
from matplotlib.colors import LinearSegmentedColormap





 
def make_Ramp( ramp_colors ): 
    from colour import Color
    from matplotlib.colors import LinearSegmentedColormap

    color_ramp = LinearSegmentedColormap.from_list( 'my_list', [ Color( c1 ).rgb for c1 in ramp_colors ] )
    return color_ramp



def eight_col(emo_dict):
  


  emo_dict = {k: v for k, v in sorted(emo_dict.items(), key=lambda item: item[1])}
  color_lists = []
  highest_emo = list(emo_dict)[-1]
  if(highest_emo in ['anger']):
    custom_ramp = make_Ramp(['#FFFFFF','#990000' ])
  elif(highest_emo in ['disgust']):
    custom_ramp = make_Ramp(['#FFFFFF','#FC0808' ]) 
  elif(highest_emo in ['fear']):
    custom_ramp = make_Ramp(['#FFFFFF','#ff5c33' ]) 
  elif(highest_emo in ['sadness']):
    custom_ramp = make_Ramp(['#FFFFFF','#FEAD39' ]) 
  elif(highest_emo in ['surprise']):
    custom_ramp = make_Ramp(['#FFFFFF','#b3ff99' ]) 
  elif(highest_emo in ['anticipation']):
    custom_ramp = make_Ramp(['#FFFFFF','#79ff4d' ]) 
  elif(highest_emo in ['trust']):
    custom_ramp = make_Ramp(['#FFFFFF','#33cc00' ]) 
  elif(highest_emo in ['joy']):
    custom_ramp = make_Ramp(['#FFFFFF','#1a6600' ]) 

  mymap =custom_ramp
  return mymap


def pos_neg_col(emo_dict):
  emo_dict = {k: v for k, v in sorted(emo_dict.items(), key=lambda item: item[1])}
  color_lists = []
  highest_emo = list(emo_dict)[-1]
  if(highest_emo in ['sadness','anger','disgust','fear']):
    
    color_lists.append(plt.cm.Reds(np.linspace(0., emo_dict[highest_emo], 128)))
  elif(highest_emo in ['joy','trust','anticipation','surprise']):
    color_lists.append(plt.cm.Greens(np.linspace(0., emo_dict[highest_emo], 128)))

  colors = np.vstack(tuple(np.array(color_lists)))
  mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
  return mymap

# emo_dict = {'acceptance': 0.037,'anger': 0.019,'boredom': 0.331,'disgust_loathing': 0.059,'fear': 0.01,'sadness': 0.585}
# emo_dict_to_c_maps(emo_dict)

def colorize(sentences, color_arrays, emo_dicts):
    # colors1 = plt.cm.Blues(np.linspace(0., 0.585, 128))
    # # mymap = colors1
    # colors2 = plt.cm.Reds(np.linspace(0, 0.331, 128))
    # colors = np.vstack((colors1, colors2))
    # colors = tuple(np.array(emo_dict_to_c_maps(emo_dict)))
    f_color_string = ''
    for words,color_array,emo_dict in zip(sentences,color_arrays,emo_dicts):
      mymap = pos_neg_col(emo_dict)
      # mymap = eight_col(emo_dict)
      template = '<span class="barcode"; style="color: black; background-color: {};display: inline-block;">{}</span>'
      colored_string = ''
      for word, color in zip(words, color_array):
          color = matplotlib.colors.rgb2hex(mymap(color)[:3])
          # print(color)
          colored_string += template.format(color, '&nbsp' + word + '&nbsp')
      f_color_string+=colored_string
    return f_color_string

# x = = 'The quick brown fox jumps over the lazy dog'.split()

def plot_paragraph(sentences,scores,emo_dict_list):
  color_arrays = []
  for each_yl in scores:
    color_array = np.array([i[0] for i in each_yl])
    color_arrays.append(color_array)
  s = colorize(sentences, color_arrays, emo_dict_list)
  # s = s[:20]+'</br>'+s[20:]
  # print(HTML(s))
  print(s)
  ss = """<div style="width: 500px;">"""+s+"""</div>"""
  print(ss)
  print(HTML(ss))

  display(HTML(ss))
  
  # or simply save in an html file and open in browser
  # with open('colorize.html', 'w') as f:
  #     f.write(s)

sentences =  [['I can', 'can kill', 'kill him', 'him and', 'and iam', 'iam so', 'so angry'],['Srilanka', 'is', 'a', 'beautiful', 'country', 'that', 'is', 'ruined', 'by', 'politicians'],['This is', 'is good.', 'good. And', 'And it', 'it is', 'is anticipated', 'anticipated totally.', 'totally. It', 'It helped', 'helped us', 'us to', 'to trust', 'trust him']]
scores =  [[[0.3744099221595089], [0.7002682285652779], [0.7608698932553926], [0.55863437774331], [0.5088178771027723], [0.4981902889500965], [0.9732918850323876]],[[0.36019073229576537], [0.3007115641872409], [0.30299748669879484], [0.39827378285158865], [0.26182668146560384], [0.303684031723546], [0.3007115641872409], [0.8428669352516664], [0.2971656227406666], [0.27657020690446865]],[[0.5399753427293827], [0.9099033747687806], [0.8461436613241728], [0.5003118554955207], [0.509717136736035], [0.6934702264520791], [0.7483976213376097], [0.7169592868690018], [0.7796496839475942], [0.6836142053451734], [0.44178627932054915], [0.6957015923763581], [0.8189624181575017]]]
emo_dict_list = [{'anger': 0.65, 'boredom': 0.322, 'disgust_loathing': 0.069},{'acceptance': 0.037,'anger': 0.019,'joy': 0.331,'disgust_loathing': 0.059,'fear': 0.01,'sadness': 0.585},{'anticipation': 0.013, 'anger': 0.007, 'trust': 0.197, 'senerity': 0.034, 'joy': 0.547, 'admire': 0.006, 'acceptance': 0.226, 'interest_vigilance': 0.011}]

plot_paragraph(sentences,scores,emo_dict_list)

<span class="barcode"; style="color: black; background-color: #fcbea5;display: inline-block;">&nbspI can&nbsp</span><span class="barcode"; style="color: black; background-color: #fb7858;display: inline-block;">&nbspcan kill&nbsp</span><span class="barcode"; style="color: black; background-color: #fb6c4c;display: inline-block;">&nbspkill him&nbsp</span><span class="barcode"; style="color: black; background-color: #fc9576;display: inline-block;">&nbsphim and&nbsp</span><span class="barcode"; style="color: black; background-color: #fca082;display: inline-block;">&nbspand iam&nbsp</span><span class="barcode"; style="color: black; background-color: #fca386;display: inline-block;">&nbspiam so&nbsp</span><span class="barcode"; style="color: black; background-color: #ec382b;display: inline-block;">&nbspso angry&nbsp</span><span class="barcode"; style="color: black; background-color: #fdc7b0;display: inline-block;">&nbspSrilanka&nbsp</span><span class="barcode"; style="color: black; background-

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # print('ime',input_mask_expanded)
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    # print('se',sum_embeddings)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:

def map_opposite_emotions(emo_dict):
  opposite_emotions = {'anticipation': 'surprise',
                       'anger': 'joy',
                       'fear': 'trust',
                       'sadness': 'joy',
                       'trust': 'fear',
                       'joy': 'sadness',
                       'surprise': 'anticipation',
                       'disgust': 'joy'
                       }
  opposed_dict = {
              'anticipation':0.0,
              'anger':0.0,
              'fear':0.0,
              'sadness':0.0,
              'trust':0.0,
              'joy':0.0,
              'surprise':0.0,
              'disgust':0.0}

  for each_key in emo_dict.keys():
    opposite_emo = opposite_emotions[each_key]
    opposed_dict[opposite_emo] = opposed_dict[opposite_emo]+emo_dict[each_key]

  # print(opposed_dict)

  return opposed_dict

map_opposite_emotions({'surprise': 0.274, 'fear': 0.3898, 'anticipation': 0.8663, 'trust': 1.0195, 'sadness': 2.8922, 'joy': 2.9685, 'anger' : 1.22, 'disgust' : 2.33})
# map_opposite_emotions({'anger': 0.353, 'disgust': 0.688})



{'anger': 0.0,
 'anticipation': 0.274,
 'disgust': 0.0,
 'fear': 1.0195,
 'joy': 6.4422,
 'sadness': 2.9685,
 'surprise': 0.8663,
 'trust': 0.3898}

In [ ]:
def get_nearest_neighbours(embeding):
    t1 = datetime.now()
    tuples = []

    # for i,row_e in df.iterrows():
        
    #     dis = cosine_similarity([row_e['embedding']], [embeding])
    #     # print([row_e['tokens'],row_d['tokens'],dis])
    #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

    indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
    for i in range(len(indexes)):
      tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]])
    # print(indexes)
    # print(distances)
    # print(tuples)
    
    s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
    neaarest_neighbs_words = []
    neaarest_neighbs_embs = []
    neaarest_neighbs_labels = []
    neaarest_neighbs_distance = []
    neaarest_neighbs_intensity_score = []
    for i,m in enumerate(s_tup[::-1]):
        # print(m)
        if(i<50):#getting the nearest 50 neighbours
            neaarest_neighbs_words.append(m[0])
            neaarest_neighbs_embs.append(m[3])
            neaarest_neighbs_labels.append(m[1])
            # distance = m[2].tolist()[0][0]
            distance = m[2]
            neaarest_neighbs_distance.append(distance)
            neaarest_neighbs_intensity_score.append(m[4])

    n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score)
    # neaarest_neighbs_words.append('sentence')
    # neaarest_neighbs_embs.append(numpy.array(embeding[0]))
    # neaarest_neighbs_labels.append('input')
    t2 = datetime.now()
    diff = t2-t1
    print('time',diff)
    # print(Counter(neaarest_neighbs_labels))
    
    #visualize the neighbours in 2d space
    # visualize_embs(neaarest_neighbs_labels,neaarest_neighbs_embs,neaarest_neighbs_words)
    return [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]

In [ ]:
def calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score):
  score_dict = {
              'anticipation':0,
              'anger':0,
              'fear':0,
              'sadness':0,
              'trust':0,
              'joy':0,
              'surprise':0,
              'disgust':0}

  #Scoring Mechanism
  for i in range(0,len(neaarest_neighbs_words)):
    # score = 50-i
    distance = 0
    if neaarest_neighbs_distance[i] == 0 :
      distance = 1
    else :
      distance = (1/neaarest_neighbs_distance[i])
    
    score = distance*(neaarest_neighbs_intensity_score[i])
    
    # print(score,neaarest_neighbs_words[i],neaarest_neighbs_labels[i])
    score_dict[neaarest_neighbs_labels[i]]=score_dict[neaarest_neighbs_labels[i]]+score

  #Normalising Mechanism
  # score_max = (len(neaarest_neighbs_words)*(len(neaarest_neighbs_words)-1))/2
  normalized_score_dict = score_dict.copy()
  # for k in score_dict.keys():
  #   if score_dict[k] ==0:
  #     continue
  #     # del normalized_score_dict[k]
  #   else:
  #     normalized_score_dict[k]
  #     # normalized_score_dict[k] = round((score_dict[k]/score_max),3)

  return normalized_score_dict

In [ ]:
def map_candidate_to_emotions(neighbour_dict,candidate_dict):
  emo_candi_dict = {}
  emo_score_dict = {}
  emo_embed_dict = {}
  neighbor_df = pd.DataFrame(neighbour_dict)
  
  dft = neighbor_df.groupby('labels')['words'].nunique().sort_values(ascending=False).reset_index(name='count')
  # print(dft)
  unique_emos = dft['labels'][:2]#mapping always to top emotion

  for each_cd in candidate_dict:
    dis_emo_dict = {}
    for each_ue in unique_emos:
      dis_list = []
      emod = neighbor_df.loc[neighbor_df['labels'] == each_ue][:50]
      for j,e_row in emod.iterrows():
        dis_list.append(cosine_similarity([e_row['embs']], [each_cd[2]]))
      dis_emo_dict[each_ue] = np.mean(dis_list) 
    emo_candi_dict[each_cd[0]]=max(dis_emo_dict, key=dis_emo_dict.get)
    emo_score_dict[each_cd[0]] = dis_emo_dict[emo_candi_dict[each_cd[0]]]
    emo_embed_dict[each_cd[0]] = each_cd[2]
    # break
  return [emo_candi_dict,emo_score_dict,emo_embed_dict]

def fix_score(current_score,in_dc,in_sc):
  if(in_dc=='B_INCR'):
    final_score = current_score+(current_score*in_sc)
  elif(in_dc=='B_DECR'):
    final_score = current_score-(current_score*in_sc)
  return final_score

print(len(intensity_modifiers))  


84


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
model = AutoModel.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [ ]:
! pip3 install syntok -q
! pip install -U sentence-transformers -q
! pip install langdetect -q

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
     |████████████████████████████████| 981 kB 5.2 MB/s 


In [ ]:
import syntok.segmenter as segmenter
import torch 
from sentence_transformers import SentenceTransformer ,  util
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from langdetect import detect
import pickle

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """


In [ ]:
# Import the sentence transformer model
model = SentenceTransformer('bhadresh-savani/distilbert-base-uncased-emotion', device = 'cuda')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bhadresh-savani_distilbert-base-uncased-emotion were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def generate_embeddings(sentences):
  embeddings = model.encode(sentences)
  embeddings=[torch.from_numpy(item) for item in embeddings]
  for i in range(len(embeddings)):
    embeddings[i] = embeddings[i].tolist()
  
  return embeddings

In [ ]:
def get_mean_pooling_emb(sentences):
  # tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-mini-finetuned-age_news-classification")
  # model = AutoModel.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
  # tokenizer = AutoTokenizer.from_pretrained("joeddav/distilbert-base-uncased-go-emotions-student")
  # model = AutoModel.from_pretrained("/content/drive/MyDrive/NLP_notebooks/Emotional_embeddings/go_model_simple")

  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
  encoded_input.to(device)
  #Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings_raw = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings_raw.tolist()

  return sentence_embeddings

In [ ]:
def check_for_negations(top_candidates):
  neg = False
  for tsp in top_candidates:
    for each_p in tsp.split(' '):
      if(each_p in negations):
        # print('profiles are negated')
        neg = True
  return neg

In [ ]:
def incremental_learning(emo_candidates, emo_score, emo_embedding):
  rows = []
  for key in emo_candidates :
    row = {}
    row['word'] = key
    row['emotion'] = emo_candidates[key]
    row['emotion-intensity-score'] = emo_score[key]
    row['embedding'] = emo_embedding[key]
    # print(row)
    if not (((df['word'] == row['word']) & (df['emotion'] == row['emotion'])).any()):
      rows.append(row)
  return rows



In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')


def emotion_candidates_recognition(sentence,window_size):
   
  tokens = sentence.split(' ')
  set_tokens = set(tokens)
  sentence_tokens = list(set_tokens)

  # print(len(tokens))
  # print(len(sentence_tokens))
  freq_dict = Counter(sentence_tokens)

  sentence_pieces = [sentence]
  for i in range(0,len(sentence_tokens)+1-window_size):
    sliding_piece = ' '.join(sentence_tokens[i:i+window_size])
    # print(sliding_piece)
    sentence_pieces.append(sliding_piece)

  # print(len(sentence_pieces))
  # sentence_emb = get_mean_pooling_emb(sentence_pieces)
  sentence_emb = generate_embeddings(sentence_pieces)
  # print("sentence emb")
  # print(len(sentence_emb[1]))
  neighbour_output = get_nearest_neighbours(sentence_emb[0])
  normalized_score_dict = neighbour_output[0]
  for key in normalized_score_dict:    
    normalized_score_dict[key] = (round((normalized_score_dict[key]/len(sentence_tokens)),4))

  neighbour_dict = neighbour_output[1]
  # print(neighbour_dict)
  

  tuples = []
  for i in range(1,len(sentence_emb)):
    sliding_piece = sentence_pieces[i]
    dis = cosine_similarity([sentence_emb[i]], [sentence_emb[0]])
    tuples.append([sliding_piece,dis,sentence_emb[i]])

  
  s_tup = sorted(tuples, key=lambda x: x[1])#sort tuples based on the cosine distance

  candidate_dict = s_tup[::-1][:5]
  # print(len(candidate_dict))
  # for each_cd in candidate_dict:
  #   print(each_cd[0])

  emo_candi_scores = map_candidate_to_emotions(neighbour_dict,candidate_dict)
  emo_candidates = emo_candi_scores[0]
  emo_scores = emo_candi_scores[1]
  emo_embeddings = emo_candi_scores[2]
  # print(emo_candidates)
  # print(emo_scores)
  # print(emo_embeddings)
  # print('*********')
  # print(df)
  rows=[]
  # rows = incremental_learning(emo_candidates,emo_scores,emo_embeddings)
  # if ! (((df['emotion'] == rows['e']) & (df['B'] == 3)).any()):
  # dataf = df.append(rows, ignore_index=False)
  # print(dataf)
  top_5_windows = [i for i in emo_candidates.keys()]
  # print('top',top_5_windows)

  # upper_threshold = int(0.33*len(sentence_tokens))
  # upper_threshold = 1
  # top_windows = [i[0] for i in s_tup[::-1][:upper_threshold]]
  # top_windows = top_5_windows[:upper_threshold]

  #intensity modifiers/negations detection

  top_windows = top_5_windows
  fixed_top_windows = []
  # print("=======")
  # print(top_windows)
  for i,emoWord in enumerate(top_windows):
      # print(i,emoWord)
      end_ind_int = sentence_tokens.index(emoWord)
      start_ind_int = end_ind_int - 3
      if start_ind_int < 0:
          start_ind_int = 0
      text_chunk_int = (' ').join(sentence_tokens[start_ind_int:end_ind_int])
      text_chunk_int = text_chunk_int.strip().lower()
      fixed_top_windows.append(text_chunk_int)

      #check_negations and intensity modifiers only in top candidate
      if(i==0):
        
        #check for intensifiers or inhibitors
        for im in intensity_modifiers:
          im_splits = im.split(':')
          int_w = im_splits[0]
          in_dc = im_splits[1]
          in_sc = float(im_splits[2])
          if(len(int_w.split())==1):
            if int_w in text_chunk_int.split():
              # print('gotcha',im)
              # print('emo',emo_candidates[emoWord])
              crnt_sc = normalized_score_dict[emo_candidates[emoWord]]
              normalized_score_dict[emo_candidates[emoWord]] = fix_score(crnt_sc,in_dc,in_sc)
              # print('fixed',normalized_score_dict)
          elif(len(int_w.split())>1):
            if int_w in text_chunk_int:
              # print('gotcha',im)
              # print('emo',emo_candidates[emoWord])
              # print(fix_score(0.5,in_dc,in_sc))
              crnt_sc = normalized_score_dict[emo_candidates[emoWord]]
              normalized_score_dict[emo_candidates[emoWord]] = fix_score(crnt_sc,in_dc,in_sc)
              # print('fixed',normalized_score_dict)


        #check nagations
        # print('check neg')
        if(check_for_negations([text_chunk_int])):
          print('Emotions are negated')
          normalized_score_dict = map_opposite_emotions(normalized_score_dict)
  # print('fixed',fixed_top_windows)
  top_5_windows = fixed_top_windows
  

  # plot_emotional_weight([i[0] for i in tuples],[i[1].tolist()[0] for i in tuples])
  
  normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
  return [normalized_score_dict,rows]
  # return [normalized_score_dict,top_5_windows[:1]]



result = (emotion_candidates_recognition("year born time became governor average increase minnesota state spending percent brought dramatically percent per year",1))
# result = (emotion_candidates_recognition("in a rare move a throng of republican students upset at ongoing harassment they face on campus took their complaints to the ultimate authority the board of trustees video below two liberals speak at the beginning of the video then republican students get into it with well thought out arguments it s the right way to protest some two dozen people many of whom students converged at the coast community college district board meeting en masse last week to voice concern at the discrimination they face at their school asking for measures to protect them from discrimination based on their political diversity and ideological beliefs as the parade of students spoke applause and high fives echoed through the room during a meeting that was charged with high emotion many were also there to protest the fact that an instructor at the school olga perez stable cox who told her human sexuality class that donald trump s election was an assault and act of terrorism recently earned the faculty of the year award shaky camera alert a liberal is the first to speak about extremist views but the people in attendance weren t having it a professor who calls our president a terrorist is pretty darned extremist the minute mark is where the republican students get to speak signs including flunk olga were held by some in the audience and others wore bright red make america great again and trump hats joshua recalde martinez a conservative student who s been targeted with online death threats and graffiti vandalism was the first of many students to speak at the meeting this is not a left or right issue this is a common sense solution to a problem which springs from extremist partisan feelings from either side of the political spectrum against the other and sadly those who advocate against these amendments seek to only allow further discrimination on college campuses throughout the district said recalde martinez president emeritus of the occ college republicans vincent wetzel president of the occ college republicans also spoke up saying he is a gay hispanic religious minority from a low income household he has never been more discriminated against for any other reason than being republican republicans are the most targeted minority on our campus and it is the duty of this board to defend minority students he said via the college fix",1))
# df = result[1]
normalized_score_dict = result[0]
# print(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
time 0:00:00.245625


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18044 non-null  int64  
 1   word                     18044 non-null  object 
 2   emotion                  18044 non-null  object 
 3   emotion-intensity-score  18044 non-null  float64
 4   embedding                18044 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 705.0+ KB


In [ ]:
def plot_emotional_weight(x,y):
  # plotting the points 
  plt.figure(figsize=(20,5))
  plt.plot(x, y)
    
  # naming the x axis
  plt.xlabel('word windows')
  # naming the y axis
  plt.ylabel('emotional contribution')
    
  # giving a title to my graph
  # plt.title('Emotion contribution o')
    
  # function to show the plot
  
  plt.show()

def check_for_negations(top_candidates):
  neg = False
  for tsp in top_candidates:
    for each_p in tsp.split(' '):
      if(each_p in negations):
        # print('profiles are negated')
        neg = True
  return neg

In [ ]:
def emotion_paragraph(text):
  window_size = 2
  sent_word_lists = []
  score_lists = []
  emo_dictionary_list = []

  sentences = nltk.tokenize.sent_tokenize(text)
  for sentence in sentences:
    sentence_tokens = sentence.split(' ')
    sent_word_lists.append(sentence_tokens)
    sentence_pieces = [sentence]
    for i in range(0,len(sentence_tokens)+1-window_size):
      sliding_piece = ' '.join(sentence_tokens[i:i+window_size])
      sentence_pieces.append(sliding_piece)

    sentence_emb = get_mean_pooling_emb(sentence_pieces)

    normalized_score_dict = get_nearest_neighbours(sentence_emb[0])[0]

    # for key in normalized_score_dict:    
    #   normalized_score_dict[key] = round((normalized_score_dict[key]/len(sentence_tokens)),4)

    tuples = []
    for i in range(1,len(sentence_emb)):
      sliding_piece = sentence_pieces[i]
      dis = cosine_similarity([sentence_emb[i]], [sentence_emb[0]])
      # print(dis)
      tuples.append([sliding_piece,dis])
    # print([i[0] for i in tuples])
    # print([i[1].tolist()[0] for i in tuples])
    
    s_tup = sorted(tuples, key=lambda x: x[1])#sort tuples based on the cosine distance
    # print(s_tup)

    top_5_windows = [i[0] for i in s_tup[::-1][:5]]
    # print(top_5_windows)

    if(window_size==1):

          # upper_threshold = int(0.33*len(sentence_tokens))
          upper_threshold = 1
          top_windows = [i[0] for i in  s_tup[::-1][:upper_threshold]]
          fixed_top_windows = []
          for emoWord in top_windows:

              end_ind_int = sentence_tokens.index(emoWord)
              start_ind_int = end_ind_int - 3
              if start_ind_int < 0:
                  start_ind_int = 0
              text_chunk_int = (' ').join(sentence_tokens[start_ind_int:end_ind_int])
              fixed_top_windows.append(text_chunk_int.strip().lower())
          print('fixed',fixed_top_windows)

          top_5_windows = fixed_top_windows
    

    if(check_for_negations(top_5_windows)):
      print('Emotions are negated')
      normalized_score_dict = map_opposite_emotions(normalized_score_dict)
    
    # normalized_score_dict = ft_to_et(normalized_score_dict)
    emo_dictionary_list.append(normalized_score_dict)


    # print('x',[i[0] for i in tuples])
    # print('y',[i[1].tolist()[0] for i in tuples])

    word_lists = sentence_pieces[1:]
    score_lists = [i[1].tolist()[0] for i in tuples]

    normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
  print([word_lists])
  print([score_lists])
  print(emo_dictionary_list)
  # plot_paragraph([word_lists],[score_lists],emo_dictionary_list)

emotion_paragraph("South Australia is the first state to have its international students arrival plan supported by the Federal Government.")

NameError: ignored

In [ ]:
import operator

def dict_to_result(emotion_dict):
  if (all(value == 0 for value in emotion_dict.values())):
    highest_8 = None
  else:
    highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
  return highest_8

result = emotion_candidates_recognition('That was good',1)
x = dict_to_result(result[0])
print("============")
print(result)
print(x)
# print(y)
# print(z)

time 0:00:00.004820
[{'anger': 0.0, 'fear': 0.0, 'sadness': 0.0, 'disgust': 0.0, 'surprise': 0.456, 'anticipation': 2.032, 'trust': 3.345, 'joy': 13.839},            word  ...                                          embedding
0      outraged  ...  [1.124068260192871, 0.7084425091743469, -0.467...
1     brutality  ...  [1.3678280115127563, 0.2582870125770569, -0.73...
2        hatred  ...  [1.1745389699935913, 0.8239633440971375, -0.41...
3       hateful  ...  [0.8323023915290833, 0.9238643646240234, -0.55...
4     terrorize  ...  [1.417137861251831, -0.09984711557626724, -1.0...
...         ...  ...                                                ...
9918   mistakes  ...  [0.8209497332572937, 0.5562782883644104, -0.70...
9919       bait  ...  [1.3154369592666626, -0.05761982873082161, -0....
9920  scoundrel  ...  [0.8288072347640991, 0.33036693930625916, -0.1...
0          That  ...  [0.5178517699241638, 0.25358113646507263, -0.8...
1           was  ...  [0.5193632245063782, -0.3461133

#Pipeline

In [ ]:
import pandas as pd

dfff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Dataset/WELFAKE_embeddings_V2.csv")

dfff.info()
dfff.drop(['input'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62611 entries, 0 to 62610
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  62611 non-null  int64 
 1   label       62611 non-null  int64 
 2   total_text  62611 non-null  object
 3   input       62611 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


In [ ]:
fake = dfff.loc[dfff['label'] == 1]
true = dfff.loc[dfff['label'] == 0]

fake = fake.sample(1000)
true = true.sample(1000)

dff=fake.append(true,ignore_index = True)
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2000 non-null   int64 
 1   label       2000 non-null   int64 
 2   total_text  2000 non-null   object
dtypes: int64(2), object(1)
memory usage: 47.0+ KB


In [ ]:
dff.head()

,Unnamed: 0,label,total_text
0,13710,1,iran may have received over billion in cash an...
1,62452,1,bake shaming now a hate crime say police octob...
2,1305,1,israel settlements legal trump aide says playi...
3,7727,1,what enthusiasm gap watching crowd go nuts as ...
4,45016,1,a trump fan picks a fight on a plane so the ca...


In [ ]:
dff = dff.dropna(subset = ['text'], how='all')

def lower(text):
  return text.lower()

dff['text'] = dff['text'].apply(lower)

In [ ]:
# Remove Duplicates

duplicate_count = len(dff['text'])-len(dff['text'].drop_duplicates())
print('Duplicate count:', duplicate_count)
print('Total records before remove duplicates:', dff.shape[0])

# drop duplicates (keep the last post of each of the duplicates)

dff = dff.drop_duplicates(subset='text', keep="first")
print('updated record count:', dff.shape[0])

Duplicate count: 0
Total records before remove duplicates: 2000
updated record count: 2000


In [ ]:
import string
import re

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  # text = text.translate(str.maketrans('', ' ', string.punctuation))
  # return text

  res = re.sub(r'[^\w\s]', ' ', text)
  return res

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

# remove multiple spaces
def remove_mul_space(text):
  return " ".join(text.split())

def clean(text):
  text = remove_urls(text)
  text = remove_nonascii(text)
  text = remove_punctuations(text)
  text = remove_num(text)
  text = remove_mul_space(text)
  return text

In [ ]:
dff['text'] = dff['text'].apply(clean)
dff = dff.reset_index(drop=True)
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             2000 non-null   int64  
 1   title                                  1986 non-null   object 
 2   text                                   2000 non-null   object 
 3   label                                  2000 non-null   int64  
 4   url_count                              2000 non-null   int64  
 5   qn_symbol                              2000 non-null   int64  
 6   len                                    2000 non-null   int64  
 7   lang                                   2000 non-null   object 
 8   num_chars                              2000 non-null   int64  
 9   num_words                              2000 non-null   int64  
 10  num_sentences                          2000 non-null   int64  
 11  word

In [ ]:
import pandas as pd

# results_df = pd.DataFrame()
# dff = pd.read_csv(path)
# print(dff.columns)
# print(dff['sentiment'].unique())

print(len(dff))
# preds = []

lexicon = []
eight_label = []
highest_8_emo_label = []
# label =[]

sentences_zz = []
t1 = datetime.now()
for i,row in dff.iterrows():
    print(i)
    # if(i>1000):continue
    row_dict = row.to_dict()
    # print()
    sentence = row['total_text']
    # print(sentence)
    # sentences_zz.append(sentence)
    # sentence_embeddings = get_mean_pooling_emb(sentence)
    # get_nearest_neighbours(sentence_embeddings)
    result = emotion_candidates_recognition(sentence,1)
    # lexicon.extend(result[1])
    label_dict = result[0]
    print(label_dict)
    eight_label.append(label_dict)
    # preds.append(pred)
    highest_eight_label = dict_to_result(label_dict)
    # fourteen_label.append(fourteen_dict)
    # eight_label.append(eight_emo_dict)
    highest_8_emo_label.append(highest_eight_label)
    # highest_14_emo_label.append(highest_fourteen_label)

# print(sentences_zz)
# # preds = []
# sentence_embeddings = get_mean_pooling_emb(sentences_zz)
# for each_emb in sentence_embeddings:
#   pred = get_nearest_neighbours([each_emb])
#   # print(pred)
#   preds.append(pred)

# df = df.append(lexicon, ignore_index=False)
out_dff = dff
# out_dff = dff[['title', 'text', 'label']]
# out_dff['14_emo_dict'] = fourteen_label

out_dff["anger"]=[i["anger"] for i in eight_label]
out_dff["anticipation"]=[i["anticipation"] for i in eight_label]
out_dff["disgust"]=[i["disgust"] for i in eight_label]
out_dff["fear"]=[i["fear"] for i in eight_label]
out_dff["joy"]=[i["joy"] for i in eight_label]
out_dff["sadness"]=[i["sadness"] for i in eight_label]
out_dff["surprise"]=[i["surprise"] for i in eight_label]
out_dff["trust"]=[i["trust"] for i in eight_label]

out_dff['8_emo_dict'] = eight_label
# out_dff['highest_14_emo_label'] = highest_14_emo_label
out_dff['highest_8_emo_label'] = highest_8_emo_label
t2 = datetime.now()
diff = t2-t1
print('time',diff)
# out_dff['predictions'] = preds

# out_dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Results/final_predictions_emotions_new.csv")



Streaming output truncated to the last 5000 lines.
{'anticipation': 0.0, 'joy': 0.0, 'trust': 0.001, 'surprise': 0.004, 'fear': 0.01, 'sadness': 0.014, 'disgust': 0.034, 'anger': 0.172}
346
time 0:00:00.005530
{'anticipation': 0.0, 'joy': 0.0, 'surprise': 0.003, 'fear': 0.003, 'trust': 0.004, 'sadness': 0.006, 'disgust': 0.032, 'anger': 0.133}
347
time 0:00:00.005431
{'anticipation': 0.0, 'trust': 0.0, 'joy': 0.0, 'sadness': 0.012, 'fear': 0.029, 'surprise': 0.04, 'disgust': 0.075, 'anger': 0.176}
348
time 0:00:00.006002
{'anticipation': 0.002, 'trust': 0.003, 'surprise': 0.006, 'joy': 0.007, 'sadness': 0.017, 'disgust': 0.019, 'anger': 0.02, 'fear': 0.033}
349
time 0:00:00.005223
{'sadness': 0.004, 'anticipation': 0.004, 'surprise': 0.01, 'fear': 0.016, 'disgust': 0.019, 'trust': 0.034, 'anger': 0.041, 'joy': 0.16}
350
time 0:00:00.004837
Emotions are negated
{'anger': 0.0, 'disgust': 0.0, 'trust': 0.004, 'anticipation': 0.006, 'joy': 0.013, 'surprise': 0.022, 'fear': 0.037, 'sadness'

In [ ]:
# out_dff['8_emo_dict'] = eight_label
# out_dff['highest_8_emo_label'] = highest_8_emo_label

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
out_dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2000 non-null   int64  
 1   label                2000 non-null   int64  
 2   total_text           2000 non-null   object 
 3   anger                2000 non-null   float64
 4   anticipation         2000 non-null   float64
 5   disgust              2000 non-null   float64
 6   fear                 2000 non-null   float64
 7   joy                  2000 non-null   float64
 8   sadness              2000 non-null   float64
 9   surprise             2000 non-null   float64
 10  trust                2000 non-null   float64
 11  8_emo_dict           2000 non-null   object 
 12  highest_8_emo_label  2000 non-null   object 
dtypes: float64(8), int64(2), object(3)
memory usage: 203.2+ KB


In [ ]:
for i in lexicon:
  print(i)

{'word': 'pedophiles', 'emotion': 'anger', 'emotion-intensity-score': 0.7654707905364962, 'embedding': [1.3457870483398438, 0.578401505947113, -0.8569102883338928, -0.5867604613304138, -0.011405415832996368, 0.1125536635518074, -0.5094768404960632, 0.8268600106239319, 0.6906914114952087, -0.2462141513824463, -0.35936352610588074, -0.26826998591423035, -1.3848414421081543, 0.06323263049125671, -0.18005186319351196, 0.04044751077890396, -0.09268567711114883, 0.24903994798660278, -0.5109257102012634, -0.44729599356651306, 0.06719423085451126, -0.4861513376235962, -0.3193460702896118, -0.027066759765148163, 0.05027191340923309, -0.45435285568237305, -0.16632577776908875, 0.1386067420244217, 0.3173300325870514, -0.38318929076194763, 1.4419727325439453, 0.18749701976776123, 0.5654518008232117, -0.02616986073553562, 0.0307018905878067, -0.21057488024234772, -0.766472578048706, 0.41619572043418884, 0.08512071520090103, 0.002090363996103406, -0.8433439135551453, 0.33132508397102356, -0.24665583

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18044 non-null  int64  
 1   word                     18044 non-null  object 
 2   emotion                  18044 non-null  object 
 3   emotion-intensity-score  18044 non-null  float64
 4   embedding                18044 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 705.0+ KB


In [ ]:
out_dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Results/final_predictions_emotions_incremental.csv")

In [ ]:
df.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity_incremental.csv")

#Creating Annoy Index

In [ ]:
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity_incremental.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18044 non-null  int64  
 1   word                     18044 non-null  object 
 2   emotion                  18044 non-null  object 
 3   emotion-intensity-score  18044 non-null  float64
 4   embedding                18044 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 705.0+ KB


In [ ]:
df.head()

,Unnamed: 0,word,emotion,emotion-intensity-score,embedding
0,0,outraged,anger,0.964,"[1.124068260192871, 0.7084425091743469, -0.467..."
1,1,brutality,anger,0.959,"[1.3678280115127563, 0.2582870125770569, -0.73..."
2,2,hatred,anger,0.953,"[1.1745389699935913, 0.8239633440971375, -0.41..."
3,3,hateful,anger,0.940,"[0.8323023915290833, 0.9238643646240234, -0.55..."
4,4,terrorize,anger,0.939,"[1.417137861251831, -0.09984711557626724, -1.0..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18044 non-null  int64  
 1   word                     18044 non-null  object 
 2   emotion                  18044 non-null  object 
 3   emotion-intensity-score  18044 non-null  float64
 4   embedding                18044 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 705.0+ KB


In [ ]:
# type(df['embedding'][10])

pandas.core.series.Series

In [ ]:
import ast

df['embedding'] = [ast.literal_eval(i) for i in df['embedding'].values.tolist()]
df.head()

,Unnamed: 0,word,emotion,emotion-intensity-score,embedding
0,0,outraged,anger,0.964,"[1.124068260192871, 0.7084425091743469, -0.467..."
1,1,brutality,anger,0.959,"[1.3678280115127563, 0.2582870125770569, -0.73..."
2,2,hatred,anger,0.953,"[1.1745389699935913, 0.8239633440971375, -0.41..."
3,3,hateful,anger,0.940,"[0.8323023915290833, 0.9238643646240234, -0.55..."
4,4,terrorize,anger,0.939,"[1.417137861251831, -0.09984711557626724, -1.0..."


In [ ]:
word_embeddings = df['embedding']

In [ ]:
print(len(word_embeddings))
print(len(word_embeddings[0]))

18044
768


In [ ]:
from annoy import AnnoyIndex

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for i in range(len(word_embeddings)):
    t.add_item(i, word_embeddings[i])

t.build(500) # 500 trees

t.save(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_incremental.ann")

True